# DIOR

In [ ]:
# -------------------
# 1) DIOR Sephora (Bazaarvoice)
# -------------------
import requests
import pandas as pd
from urllib.parse import urljoin

sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP440962&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
sephora_json = requests.get(sephora_url, timeout=60).json()
sephora_reviews = sephora_json.get("Results", [])
products_info = (sephora_json.get("Includes") or {}).get("Products", {})


dior_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": 'Dior Forever Skin Glow',
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "77 CAD",   # <-- added
} for r in sephora_reviews])


# -------------------
# 2) DIOR Shoppers (PowerReviews)
# -------------------
shoppers_url = (
    "https://display.powerreviews.com/m/867678/l/all/product/3348901611282/reviews"
    "?apikey=f40b12ae-7efa-423a-8f12-cb902debf1c0&_noconfig=true&page_locale=en_CA"
)
shoppers_json = requests.get(shoppers_url, timeout=60).json()
results = shoppers_json.get("results", [])
block = results[0] if results else {}
shoppers_reviews = block.get("reviews", [])

df_shoppers_raw = pd.json_normalize(shoppers_reviews, sep=".")

shoppers_time = pd.to_datetime(
    pd.to_numeric(df_shoppers_raw.get("details.created_date"), errors="coerce"),
    unit="ms", errors="coerce", utc=True
)

rating_series = pd.Series([None] * len(df_shoppers_raw))
if "metrics.rating" in df_shoppers_raw:
    rating_series = df_shoppers_raw["metrics.rating"]
elif "details.rating" in df_shoppers_raw:
    rating_series = df_shoppers_raw["details.rating"]


dior_shoppers = pd.DataFrame({
    "source": "powerreviews",
    "retailer": "shoppers_ca",
    "product_id": block.get("page_id"),
    "product_name": 'Dior Forever Skin Glow',
    "rating": pd.to_numeric(rating_series, errors="coerce"),
    "submission_time": shoppers_time,
    "user_nickname": df_shoppers_raw.get("details.nickname"),
    "review_text": df_shoppers_raw.get("details.comments"),
    "price": "77 CAD",   # <-- added
})

# -------------------
# 3) DIOR Ulta (PowerReviews, multiple pages)
# -------------------
base = "https://display.powerreviews.com"
api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"
url = f"{base}/m/6406/l/en_US/product/pimprod2036824/reviews?apikey={api_key}&_noconfig=true&page_locale=en_US"

ulta_rows = []
headers = {"User-Agent": "Mozilla/5.0"}
max_pages = 20

page_count = 0
while True:
    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results")
    block = results if isinstance(results, dict) else (results[0] if results else {})
    reviews = block.get("reviews") or []
    if not reviews:
        break

    df_ulta_raw = pd.json_normalize(reviews, sep=".")
    n_ulta = len(df_ulta_raw)

    ul_num = pd.to_numeric(df_ulta_raw.get("details.created_date", pd.Series([None]*n_ulta)), errors="coerce")
    ul_ms = pd.to_datetime(ul_num.where(ul_num >= 1e12), unit="ms", errors="coerce", utc=True)
    ul_s  = pd.to_datetime(ul_num.where(ul_num <  1e12), unit="s",  errors="coerce", utc=True)
    ul_time = ul_ms.combine_first(ul_s)

    ul_disp = df_ulta_raw.get("details.nickname", pd.Series([None]*n_ulta))
    if "reviewer.display_name" in df_ulta_raw:
        ul_disp = df_ulta_raw["reviewer.display_name"].combine_first(ul_disp)
    if "reviewer.name" in df_ulta_raw:
        ul_disp = ul_disp.combine_first(df_ulta_raw["reviewer.name"])

    ul_text = df_ulta_raw.get("details.comments", pd.Series([None]*n_ulta)).combine_first(
        df_ulta_raw.get("review_text", pd.Series([None]*n_ulta))
    )

    if "metrics.rating" in df_ulta_raw:
        ul_rating = df_ulta_raw["metrics.rating"]
    elif "details.rating" in df_ulta_raw:
        ul_rating = df_ulta_raw["details.rating"]
    else:
        ul_rating = pd.Series([None]*n_ulta)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "ulta_us",
        "product_id": "pimprod2036824",
        "product_name": "Dior Forever Skin Glow",
        "rating": pd.to_numeric(ul_rating, errors="coerce"),
        "submission_time": ul_time,
        "user_nickname": ul_disp,
        "review_text": ul_text,
        "price": "60 USD",   # <-- added here
    })
    ulta_rows.append(df_tmp)

    page_count += 1
    if page_count >= max_pages:
        break

    paging = js.get("paging") or block.get("paging") or {}
    next_rel = paging.get("next_page_url")
    if not next_rel:
        break
    url = urljoin(base, next_rel)
    if "apikey=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}apikey={api_key}"
    if "_noconfig=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}_noconfig=true"

dior_ulta = pd.concat(ulta_rows, ignore_index=True)


# -------------------
# COMBINE ALL (Dior)
# -------------------
dior_all = pd.concat([dior_sephora, dior_shoppers, dior_ulta], ignore_index=True)


print(dior_all["retailer"].value_counts())


dior_all.head()

retailer
sephora_ca     100
ulta_us        100
shoppers_ca      3
Name: count, dtype: int64


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2558328,Dior Forever Skin Glow,1,2025-10-20 01:06:41+00:00,Alaa71,they sold items used before never buy online i...,77 CAD
1,bazaarvoice,sephora_ca,2558047,Dior Forever Skin Glow,1,2025-09-30 23:06:18+00:00,BrettC,"I was colour matched in the store, but I thoug...",77 CAD
2,bazaarvoice,sephora_ca,2558062,Dior Forever Skin Glow,5,2025-09-16 22:58:22+00:00,computerface21,"WOW. Great coverage but doesn't look ""cakey"" ...",77 CAD
3,bazaarvoice,sephora_ca,2557999,Dior Forever Skin Glow,5,2025-09-16 02:56:02+00:00,PandaMochi,I think I may have found my holy grail foundat...,77 CAD
4,bazaarvoice,sephora_ca,2558054,Dior Forever Skin Glow,5,2025-08-16 03:04:05+00:00,pbutter234,Absolutely loving it. It gives such a smooth a...,77 CAD


# NARS

In [ ]:
import requests
import pandas as pd

base = "https://display.powerreviews.com/m/6406/l/en_US/product/pimprod2030141/reviews"
api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"

nars_ulta_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop over offsets (0, 5, 10, ... 100)
for offset in range(0, 96, 5):
    url = (
        f"{base}?paging.from={offset}&paging.size=5"
        f"&filters=&search=&sort=Newest&image_only=false"
        f"&page_locale=en_US&native_only=true&_noconfig=true&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results")
    block = results if isinstance(results, dict) else (results[0] if results else {})
    reviews = block.get("reviews") or []
    if not reviews:
        break  # stop if no more reviews

    df_raw = pd.json_normalize(reviews, sep=".")

    # handle time (ms or s)
    ul_num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ul_ms = pd.to_datetime(ul_num.where(ul_num >= 1e12), unit="ms", errors="coerce", utc=True)
    ul_s  = pd.to_datetime(ul_num.where(ul_num <  1e12), unit="s",  errors="coerce", utc=True)
    ul_time = ul_ms.combine_first(ul_s)

    # reviewer name
    ul_disp = df_raw.get("details.nickname", pd.Series([None]*len(df_raw)))
    if "reviewer.display_name" in df_raw:
        ul_disp = df_raw["reviewer.display_name"].combine_first(ul_disp)
    if "reviewer.name" in df_raw:
        ul_disp = ul_disp.combine_first(df_raw["reviewer.name"])

    # review text
    ul_text = df_raw.get("details.comments", pd.Series([None]*len(df_raw))).combine_first(
        df_raw.get("review_text", pd.Series([None]*len(df_raw)))
    )

    # rating
    if "metrics.rating" in df_raw:
        ul_rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        ul_rating = df_raw["details.rating"]
    else:
        ul_rating = pd.Series([None]*len(df_raw))

    nars_df = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "ulta_us",
        "product_id": "pimprod2030141",
        "product_name": "NARS Light Reflecting Foundation",
        "rating": pd.to_numeric(ul_rating, errors="coerce"),
        "submission_time": ul_time,
        "user_nickname": ul_disp,
        "review_text": ul_text,
    })

    nars_ulta_rows.append(nars_df)

# Final DataFrame for NARS Ulta reviews
nars_ulta = pd.concat(nars_ulta_rows, ignore_index=True)

print("Reviews fetched from Ulta (NARS):", len(nars_ulta))
nars_ulta.head()


Reviews fetched from Ulta (NARS): 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text
0,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-10-23 17:23:25.363000+00:00,Judith,Stays on all day\nDoes not have a yellow tint....
1,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,1,2025-10-21 22:37:39.567000+00:00,Stephanie,"I made the purchase, which was a splurge, base..."
2,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-10-19 18:46:55.632000+00:00,Becky,Can't say enough good things.
3,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-10-19 01:00:20.165000+00:00,Aly?,I absolutely love this foundation. It has been...
4,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,2,2025-10-08 15:26:06.658000+00:00,Jane,"I've been buying this for years, I guess I got..."


In [ ]:
# Example: Add a new column named 'new_column' with some sample values
nars_ulta['price'] = '54 USD'

# Display the updated DataFrame
display(nars_ulta.head())

,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-10-23 17:23:25.363000+00:00,Judith,Stays on all day\nDoes not have a yellow tint....,54 USD
1,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,1,2025-10-21 22:37:39.567000+00:00,Stephanie,"I made the purchase, which was a splurge, base...",54 USD
2,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-10-19 18:46:55.632000+00:00,Becky,Can't say enough good things.,54 USD
3,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-10-19 01:00:20.165000+00:00,Aly?,I absolutely love this foundation. It has been...,54 USD
4,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,2,2025-10-08 15:26:06.658000+00:00,Jane,"I've been buying this for years, I guess I got...",54 USD


In [ ]:
nars_ulta['retailer'].value_counts()

,count
retailer,
ulta_us,100


In [ ]:
# -------------------
# NARS Sephora (Bazaarvoice)
# -------------------
nars_sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP479338&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
nars_sephora_json = requests.get(nars_sephora_url, timeout=60).json()
nars_sephora_reviews = nars_sephora_json.get("Results", [])
products_info = (nars_sephora_json.get("Includes") or {}).get("Products", {})

nars_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": "NARS Light Reflecting Foundation",
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "73 CAD",   # <-- corrected price
} for r in nars_sephora_reviews])

print("NARS Sephora — number of reviews:", len(nars_sephora))
nars_sephora.head()


NARS Sephora — number of reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2514552,NARS Light Reflecting Foundation,4,2025-10-20 14:35:26+00:00,exquisitelysass,This is a good foundation but it’s definitely ...,73 CAD
1,bazaarvoice,sephora_ca,2514727,NARS Light Reflecting Foundation,1,2025-10-17 13:02:39+00:00,momo2b74,"The product initially felt good, giving my ski...",73 CAD
2,bazaarvoice,sephora_ca,2514651,NARS Light Reflecting Foundation,2,2025-10-16 23:02:36+00:00,pretoo,I basically bought it to cover my pigmentation...,73 CAD
3,bazaarvoice,sephora_ca,2514735,NARS Light Reflecting Foundation,2,2025-10-16 14:37:15+00:00,MikNicole,Decent coverage - made me break out. Not purch...,73 CAD
4,bazaarvoice,sephora_ca,2842292,NARS Light Reflecting Foundation,5,2025-10-15 23:33:29+00:00,Tempriss,"I was helped in the store by the manager, who ...",73 CAD


In [ ]:
# -------------------
# NARS Shoppers (PowerReviews, multiple pages)
# -------------------
import requests
import pandas as pd

base = "https://display.powerreviews.com/m/867678/l/all/product/194251150192/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

nars_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop pages in steps of 10 reviews (paging.size=10)
for offset in range(0, 96, 10):
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else (results if isinstance(results, dict) else {})
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # Convert timestamps (ms or sec)
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # Reviewer name
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # Review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # Rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "NARS Light Reflecting Foundation",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "73 CAD",   # <-- correct price
    })
    nars_shoppers_rows.append(df_tmp)

nars_shoppers = pd.concat(nars_shoppers_rows, ignore_index=True)

print("NARS Shoppers — number of reviews:", len(nars_shoppers))
nars_shoppers.head()


NARS Shoppers — number of reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,3,2025-10-20 18:03:31.509000+00:00,Lori,Wrong color so it was really difficult to revi...,73 CAD
1,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-10-17 16:56:42.109000+00:00,Luiza,Tiny amount covers the whole face disappearing...,73 CAD
2,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,1,2025-10-10 20:48:17.336000+00:00,Pardis,I used to love this product but this time I no...,73 CAD
3,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-10-03 17:35:46.607000+00:00,Brenda,I am 68 y/o and have had a difficult time find...,73 CAD
4,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-09-30 17:22:46.193000+00:00,Yasmeen,I absolutely love this foundation. For one it ...,73 CAD


In [ ]:
# -------------------
# COMBINE ALL (NARS)
# -------------------
nars_all = pd.concat([nars_sephora, nars_shoppers, nars_ulta], ignore_index=True)

print("Rows saved (NARS):", len(nars_all))
print(nars_all.groupby(["retailer", "price"]).size().reset_index(name="num_reviews"))

nars_all.head()


Rows saved (NARS): 300
      retailer   price  num_reviews
0   sephora_ca  73 CAD          100
1  shoppers_ca  73 CAD          100
2      ulta_us  54 USD          100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2514552,NARS Light Reflecting Foundation,4,2025-10-20 14:35:26+00:00,exquisitelysass,This is a good foundation but it’s definitely ...,73 CAD
1,bazaarvoice,sephora_ca,2514727,NARS Light Reflecting Foundation,1,2025-10-17 13:02:39+00:00,momo2b74,"The product initially felt good, giving my ski...",73 CAD
2,bazaarvoice,sephora_ca,2514651,NARS Light Reflecting Foundation,2,2025-10-16 23:02:36+00:00,pretoo,I basically bought it to cover my pigmentation...,73 CAD
3,bazaarvoice,sephora_ca,2514735,NARS Light Reflecting Foundation,2,2025-10-16 14:37:15+00:00,MikNicole,Decent coverage - made me break out. Not purch...,73 CAD
4,bazaarvoice,sephora_ca,2842292,NARS Light Reflecting Foundation,5,2025-10-15 23:33:29+00:00,Tempriss,"I was helped in the store by the manager, who ...",73 CAD


In [ ]:
nars_all['retailer'].value_counts()

,count
retailer,
sephora_ca,100
shoppers_ca,100
ulta_us,100


In [ ]:
dior_all['retailer'].value_counts()

,count
retailer,
sephora_ca,100
ulta_us,100
shoppers_ca,3


# ESTEE LAUDER

In [ ]:
import requests
import pandas as pd

# -------------------
# Estée Lauder Double Wear – Shoppers (PowerReviews)
# -------------------
base = "https://display.powerreviews.com/m/867678/l/all/product/027131973195/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

estee_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop through review pages
for offset in range(0, 96, 10):  # paging.size=10, step by 10
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else (results if isinstance(results, dict) else {})
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # Time conversion
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # Reviewer
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # Review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # Rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "Estée Lauder Double Wear",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "67 CAD",   # 🔹 update to the real price at Shoppers
    })
    estee_shoppers_rows.append(df_tmp)

# Final shoppers DataFrame
estee_shoppers = pd.concat(estee_shoppers_rows, ignore_index=True)

print("Estée Lauder Double Wear — Shoppers reviews:", len(estee_shoppers))
estee_shoppers.head(6)



Estée Lauder Double Wear — Shoppers reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-10-23 12:28:20.639000+00:00,Libby,I switched to Estée Lauder probably 10 years a...,67 CAD
1,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-10-15 13:01:40.471000+00:00,Shezradioactive,Even though I struggled finding my right shade...,67 CAD
2,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-10-12 19:47:14.969000+00:00,JudyR,My sister raved about this foundation for year...,67 CAD
3,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-09-27 14:55:00.402000+00:00,Donna,I don't usually wear a matte foundation but de...,67 CAD
4,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-09-25 23:59:52.826000+00:00,Misslindsey12,I have been using this foundation for over 10 ...,67 CAD
5,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,4,2025-09-17 17:24:18.152000+00:00,Nusha,"It has great coverage, but be sure to shake it...",67 CAD


In [ ]:
import requests
import pandas as pd
from urllib.parse import urljoin

# -------------------
# Estée Lauder Double Wear – Ulta (PowerReviews, multiple pages)
# -------------------
base = "https://display.powerreviews.com"
api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"
# product path from your URL
path = "/m/6406/l/en_US/product/xlsImpprod14641507/reviews"
url = f"{base}{path}?apikey={api_key}&_noconfig=true"

ulta_rows = []
headers = {"User-Agent": "Mozilla/5.0"}
max_pages = 20

page_count = 0
while True:
    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results")
    block = results if isinstance(results, dict) else (results[0] if results else {})
    reviews = block.get("reviews") or []
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # convert created_date
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # user nickname
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "ulta_us",
        "product_id": block.get("page_id", "xlsImpprod14641507"),
        "product_name": "Estée Lauder Double Wear",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "52 USD",   # set your Ulta price
    })
    ulta_rows.append(df_tmp)

    page_count += 1
    if page_count >= max_pages:
        break

    paging = js.get("paging") or block.get("paging") or {}
    next_rel = paging.get("next_page_url")
    if not next_rel:
        break
    url = urljoin(base, next_rel)
    if "apikey=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}apikey={api_key}"
    if "_noconfig=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}_noconfig=true"

estee_ulta = pd.concat(ulta_rows, ignore_index=True)

print("Estée Lauder Double Wear — Ulta reviews:", len(estee_ulta))
estee_ulta.head()

Estée Lauder Double Wear — Ulta reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-10-21 15:35:07.977000+00:00,Sue,"FINALLY, I FOUND IT. After many trial and err...",52 USD
1,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,1,2025-10-21 15:28:00.354000+00:00,lovesmakeup25,I've been using this foundation for almost 20 ...,52 USD
2,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,4,2025-10-20 22:02:37.779000+00:00,Rachel,I have never found a better shade match and fe...,52 USD
3,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-10-20 18:44:22.286000+00:00,Marisol,This foundation is not full coverage BUT it is...,52 USD
4,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-10-18 16:14:43.166000+00:00,Lisa,Best match to my skin and color will definitel...,52 USD


In [ ]:
estee_ulta['retailer'].value_counts()

,count
retailer,
ulta_us,100


In [ ]:
# -------------------
# Estée Lauder Double Wear – Sephora (Bazaarvoice)
# -------------------
estee_sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP378284&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
estee_sephora_json = requests.get(estee_sephora_url, timeout=60).json()
estee_sephora_reviews = estee_sephora_json.get("Results", [])
products_info = (estee_sephora_json.get("Includes") or {}).get("Products", {})

estee_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": "Estée Lauder Double Wear",
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "70.5 CAD",  # <— update to correct Sephora price
} for r in estee_sephora_reviews])

print("Estée Lauder Double Wear — Sephora reviews:", len(estee_sephora))
estee_sephora.head()


Estée Lauder Double Wear — Sephora reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2052835,Estée Lauder Double Wear,4,2025-10-26 16:43:17+00:00,Anahy1217,My all time favorite foundation. As an oily gi...,70.5 CAD
1,bazaarvoice,sephora_ca,1469782,Estée Lauder Double Wear,5,2025-10-26 05:31:53+00:00,None,This covers everything! Will continue to use t...,70.5 CAD
2,bazaarvoice,sephora_ca,1465293,Estée Lauder Double Wear,5,2025-10-25 13:31:47+00:00,BecksCurley,The BEST long lasting foundation. I’ve been bu...,70.5 CAD
3,bazaarvoice,sephora_ca,2052835,Estée Lauder Double Wear,5,2025-10-18 18:16:44+00:00,miaorrick,This is the best foundation i’ve ever used! It...,70.5 CAD
4,bazaarvoice,sephora_ca,1465186,Estée Lauder Double Wear,5,2025-10-18 13:52:31+00:00,ValentynaHayda,I love this foundation so much. It’s the best ...,70.5 CAD


In [ ]:
# -------------------
# COMBINE ALL (Estée Lauder Double Wear)
# -------------------
estee_all = pd.concat(
    [estee_sephora, estee_shoppers, estee_ulta],
    ignore_index=True
)

estee_all.head()

,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2052835,Estée Lauder Double Wear,4,2025-10-26 16:43:17+00:00,Anahy1217,My all time favorite foundation. As an oily gi...,70.5 CAD
1,bazaarvoice,sephora_ca,1469782,Estée Lauder Double Wear,5,2025-10-26 05:31:53+00:00,None,This covers everything! Will continue to use t...,70.5 CAD
2,bazaarvoice,sephora_ca,1465293,Estée Lauder Double Wear,5,2025-10-25 13:31:47+00:00,BecksCurley,The BEST long lasting foundation. I’ve been bu...,70.5 CAD
3,bazaarvoice,sephora_ca,2052835,Estée Lauder Double Wear,5,2025-10-18 18:16:44+00:00,miaorrick,This is the best foundation i’ve ever used! It...,70.5 CAD
4,bazaarvoice,sephora_ca,1465186,Estée Lauder Double Wear,5,2025-10-18 13:52:31+00:00,ValentynaHayda,I love this foundation so much. It’s the best ...,70.5 CAD


# ARMANI

In [ ]:
import requests
import pandas as pd

# -------------------
# Armani Luminous Foundation — Shoppers (PowerReviews)
# -------------------
base = "https://display.powerreviews.com/m/867678/l/all/product/3605521752710/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

armani_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop in steps of 10 (paging.size=10)
for offset in range(0, 96, 10):
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else (results if isinstance(results, dict) else {})
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # parse times
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # reviewer nickname / fallback
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "Armani Luminous Foundation",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "85 CAD",  # <-- fill in the correct Shoppers price in CAD
    })

    armani_shoppers_rows.append(df_tmp)

armani_shoppers = pd.concat(armani_shoppers_rows, ignore_index=True)
armani_shoppers.head()


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-06-07 13:23:34+00:00,Erincraw,this is my favorite glow foundation and I love...,85 CAD
1,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-06-06 00:38:33+00:00,lizsingle,I have been using luminous silk natural founda...,85 CAD
2,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-06-03 18:23:24+00:00,gail,the best! i have been using it for years. it i...,85 CAD
3,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-05-28 19:18:39+00:00,Lorelei,Simply GORGEOUS. An indulgent sensation that c...,85 CAD
4,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-05-25 20:55:53+00:00,Nellie,I used Crema Nuda for years and am sorry to se...,85 CAD


In [ ]:
import requests
import pandas as pd

# -------------------
# Armani Luminous Foundation — Sephora (Bazaarvoice)
# -------------------
armani_sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP393401&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
armani_sephora_json = requests.get(armani_sephora_url, timeout=60).json()
armani_sephora_reviews = armani_sephora_json.get("Results", [])
products_info = (armani_sephora_json.get("Includes") or {}).get("Products", {})

armani_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": "Armani Luminous Foundation",
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "85 CAD",  # <-- fill in the actual price here
} for r in armani_sephora_reviews])

print("Armani Luminous — Sephora reviews:", len(armani_sephora))
armani_sephora.head()


Armani Luminous — Sephora reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,1359579,Armani Luminous Foundation,4,2025-10-26 22:29:02+00:00,KennaPaige,"I love this foundation so much, I am a normal ...",85 CAD
1,bazaarvoice,sephora_ca,1359611,Armani Luminous Foundation,5,2025-10-24 18:53:04+00:00,MaryKolos,I’ve been using this foundation for weeks and ...,85 CAD
2,bazaarvoice,sephora_ca,1359595,Armani Luminous Foundation,5,2025-10-21 23:28:32+00:00,DaniTro,It sinks in really well on my dry skin. I don’...,85 CAD
3,bazaarvoice,sephora_ca,1491356,Armani Luminous Foundation,1,2025-10-20 02:26:37+00:00,Munchkin11,I’m TIRED of looking for foundations/concealer...,85 CAD
4,bazaarvoice,sephora_ca,1359611,Armani Luminous Foundation,5,2025-10-19 17:06:25+00:00,aj2000queenie,I’ve been using this for a long time. I’ve tri...,85 CAD


In [ ]:
# -------------------
# COMBINE ALL (Armani Luminous Foundation)
# -------------------
armani_all = pd.concat(
    [armani_sephora, armani_shoppers],   # add armani_ulta here later if you have it
    ignore_index=True
)


In [ ]:
# -------------------
# COMBINE ALL BRANDS
# -------------------

all_reviews = pd.concat(
    [dior_all, nars_all, estee_all, armani_all],
    ignore_index=True
)

print("Total rows (all brands):", len(all_reviews))


Total rows (all brands): 1003


In [ ]:
all_reviews['product_name'].value_counts()

,count
product_name,
NARS Light Reflecting Foundation,300
Estée Lauder Double Wear,300
Dior Forever Skin Glow,203
Armani Luminous Foundation,200


In [ ]:
all_reviews.to_csv('all_reviews.csv', index=False)

print("✅ File saved as all_reviews.csv")

✅ File saved as all_reviews.csv
